In [1]:
import os

import pandas as pd

In [2]:
cwd = os.getcwd()   #Current working directory

In [3]:
gravity_dict = {}
time_periods = ['2000_2004', '2005_2009', '2010_2014', '2015_2019']
for item in time_periods:
    gravity_dict[item] = pd.read_csv(os.path.join(cwd, 'data', f'release_2.1_{item}.csv'))

df_trade_balance = pd.read_csv(os.path.join(cwd, 'data', 'DOT_07-12-2022 22-28-04-74_timeSeries.csv'))

C:\Users\HydraJ\AppData\Local\Temp\ipykernel_13004\2814544786.py:6: DtypeWarning: Columns (31,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trade_balance = pd.read_csv(os.path.join(cwd, 'data', 'DOT_07-12-2022 22-28-04-74_timeSeries.csv'))


In [4]:
df_trade_balance = df_trade_balance[(df_trade_balance['Indicator Name'].isin([
    'Goods, Value of Exports, Free on board (FOB), US Dollars',
    'Goods, Value of Imports, Free on board (FOB), US Dollars'])) & \
    (df_trade_balance['Attribute'] == 'Value')
]

In [5]:
df_trade_balance.insert(0, 'Index_col', df_trade_balance['Country Name'] + ' - ' + df_trade_balance['Counterpart Country Name'])

In [6]:
years = df_trade_balance.columns[-20: -1]

df_trade_balance_final = df_trade_balance.pivot(
    index = 'Index_col',
    columns = 'Indicator Name',
    values = '2000'
)
df_trade_balance_final.columns = df_trade_balance_final.columns.get_level_values(0)
df_trade_balance_final.index.name = None
df_trade_balance_final.columns.name = None
df_trade_balance_final.index += ' - 2000'

for year in years:
    df_temp = df_trade_balance.pivot(
        index = 'Index_col',
        columns = 'Indicator Name',
        values = year
    )
    df_temp.columns = df_temp.columns.get_level_values(0)
    df_temp.index.name = None
    df_temp.columns.name = None
    df_temp.index += f' - {year}'
    df_trade_balance_final = pd.concat([df_trade_balance_final, df_temp])

df_trade_balance_final.reset_index(inplace = True)

In [7]:
df_gravity = gravity_dict['2000_2004'].copy()
df_gravity.index = df_gravity['country_o'] + ' - ' + df_gravity['country_d'] + ' - ' + df_gravity['year'].astype(str)
df_gravity.drop(df_gravity.columns[:7], axis = 1, inplace = True)

for item in time_periods[1:]:
    df_temp = gravity_dict['2000_2004'].copy()
    df_temp.index = df_temp['country_o'] + ' - ' + df_temp['country_d'] + ' - ' + df_temp['year'].astype(str)
    df_temp.drop(df_temp.columns[:7], axis = 1, inplace = True)
    df_gravity = pd.concat([df_gravity, df_temp])

df_gravity.reset_index(inplace = True)

In [47]:
df_merge = df_trade_balance_final.merge(df_gravity, how="inner")
df = df_merge[~df_merge.isna()].drop_duplicates()
df = df.rename(columns={"Goods, Value of Exports, Free on board (FOB), US Dollars": "Exports_FOB", "Goods, Value of Imports, Free on board (FOB), US Dollars": "Imports_FOB"})

In [48]:
df = df[~(df[["Exports_FOB", "Imports_FOB"]].isna()).all(axis=1)]
df[["Exports_FOB", "Imports_FOB"]] = df[["Exports_FOB", "Imports_FOB"]].fillna(0)

In [51]:
use_cols = ['index', 'Exports_FOB', 'Imports_FOB', 'colony_of_destination_ever',
       'colony_of_origin_ever', 'colony_ever', 'common_colonizer',
       'common_legal_origin', 'contiguity', 'distance', 'member_gatt_o',
       'member_wto_o', 'member_eu_o', 'member_gatt_d', 'member_wto_d',
       'member_eu_d', 'member_gatt_joint', 'member_wto_joint',
       'member_eu_joint', 'landlocked_o',
       'island_o', 'region_o', 'landlocked_d', 'island_d', 'region_d',
       'agree_pta_goods', 'agree_pta_services', 'agree_fta', 'agree_eia',
       'agree_cu', 'agree_psa', 'agree_fta_eia', 'agree_cu_eia', 'agree_pta',
       'gdp_pwt_const_d', 'gdp_pwt_const_o',
       'pop_d', 'pop_o', 
       'common_language', 'polity_o', 'polity_d', 'sanction_threat',
       'sanction_threat_trade', 'sanction_imposition',
       'sanction_imposition_trade']
df = df[use_cols]

In [ ]:
feature_cols = ['colony_of_destination_ever',
       'colony_of_origin_ever', 'colony_ever', 'common_colonizer',
       'common_legal_origin', 'contiguity', 'distance', 'member_gatt_o',
       'member_wto_o', 'member_eu_o', 'member_gatt_d', 'member_wto_d',
       'member_eu_d', 'member_gatt_joint', 'member_wto_joint',
       'member_eu_joint', 'landlocked_o',
       'island_o', 'region_o', 'landlocked_d', 'island_d', 'region_d',
       'agree_pta_goods', 'agree_pta_services', 'agree_fta', 'agree_eia',
       'agree_cu', 'agree_psa', 'agree_fta_eia', 'agree_cu_eia', 'agree_pta',
       'gdp_pwt_const_d', 'gdp_pwt_const_o',
       'pop_d', 'pop_o', 
       'common_language', 'polity_o', 'polity_d', 'sanction_threat',
       'sanction_threat_trade', 'sanction_imposition',
       'sanction_imposition_trade']
target_col = ["Trade_Volume"]
df = df[use_cols]

In [59]:
df = df[~(df.isna()).any(axis=1)]
df["Trade_Volume"] = df["Exports_FOB"].astype(float) + df["Imports_FOB"].astype(float)

In [61]:
df

,index,Exports_FOB,Imports_FOB,colony_of_destination_ever,colony_of_origin_ever,colony_ever,common_colonizer,common_legal_origin,contiguity,distance,...,pop_d,pop_o,common_language,polity_o,polity_d,sanction_threat,sanction_threat_trade,sanction_imposition,sanction_imposition_trade,Trade_Volume
20,Albania - Australia - 2000,22272,0,0.0,0.0,0.0,0.0,0.0,0.0,15153.853236,...,19.065838,3.121970,1,5.0,10.0,0.0,0.0,0.0,0.0,22272.0
24,Albania - Austria - 2000,1767571,0,0.0,0.0,0.0,0.0,0.0,0.0,837.797060,...,8.069276,3.121970,0,5.0,10.0,0.0,0.0,0.0,0.0,1767571.0
72,Albania - Bulgaria - 2000,80864,0,0.0,0.0,0.0,1.0,1.0,0.0,459.123844,...,7.997787,3.121970,1,5.0,8.0,0.0,0.0,0.0,0.0,80864.0
88,Albania - Canada - 2000,388,0,0.0,0.0,0.0,0.0,0.0,0.0,7802.350253,...,30.735773,3.121970,0,5.0,10.0,0.0,0.0,0.0,0.0,388.0
116,Albania - Denmark - 2000,2689506,0,0.0,0.0,0.0,0.0,0.0,0.0,1735.343668,...,5.341194,3.121970,0,5.0,10.0,0.0,0.0,0.0,0.0,2689506.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374860,Zambia - Thailand - 2004,1108425,0,0.0,0.0,0.0,0.0,0.0,0.0,8551.398246,...,65.002228,11.731746,1,5.0,9.0,0.0,0.0,0.0,0.0,1108425.0
374888,Zambia - Uganda - 2004,534028,0,0.0,0.0,0.0,0.0,0.0,0.0,1722.325257,...,27.568436,11.731746,1,5.0,-4.0,0.0,0.0,0.0,0.0,534028.0
374896,Zambia - United Arab Emirates - 2004,1063407,0,0.0,0.0,0.0,1.0,1.0,0.0,5239.456484,...,4.087931,11.731746,1,5.0,-8.0,0.0,0.0,0.0,0.0,1063407.0
374900,Zambia - United Kingdom - 2004,274985837,0,1.0,0.0,1.0,0.0,1.0,0.0,7945.633640,...,59.884129,11.731746,1,5.0,10.0,0.0,0.0,0.0,0.0,274985837.0
